# Analisis de datos[Valuaciones]

In [5]:
import os, json
import numpy as np
from sklearn.neighbors.kde import KernelDensity
import scipy.stats as st
import statsmodels.datasets
import plotly.plotly as py
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from scipy.stats import norm
import pandas as pd
import warnings
pd.set_option('precision',2)

In [6]:
def reader_json_val(path_json):
    path_to_json=(r'C:\Users\luism\OneDrive\Escritorio\data')
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    num = (len(json_files))
    diccionario = []
    for i in range(num-1):
        json_filesx=pd.read_json(r'C:\Users\luism\OneDrive\Escritorio\data'+'/'+json_files[i+0])
        diccionario.append(json_filesx)
    diccionario=pd.concat(diccionario,axis=1)
    dicci=dict()
    for icount in range(len(diccionario.columns)):
            tempkey='resultado_'+str(icount)
            dicci[icount+1]=diccionario[tempkey]['valuacion']
            var=pd.DataFrame.from_dict(dicci.items())
            var=var[1]
    return(var)
var=reader_json_val(r'C:\Users\luism\OneDrive\Escritorio\data')

In [12]:
def best_fit_distribution(data, bins=200):    
    """Model data by finding best fit distribution to data"""
    '''O. Jaramillo Zuloaga'''
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0
    
    DISTRIBUTIONS = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    ]
    iCount =0    
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf    
    for distribution in DISTRIBUTIONS:        
        try:                
            warnings.filterwarnings('ignore')   
            params = distribution.fit(data)
            arg = params[:-2]
            loc = params[-2]
            scale = params[-1]
            pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
            iCount+=1
            sse = np.sum(np.power(y - pdf, 2.0))                                
            if best_sse > sse:
                best_distribution = distribution
                best_params = params
                best_sse = sse
        except Exception:
            pass 
    return (best_distribution,best_params)
best_dist=best_fit_distribution(var)

In [16]:
def dist(best_dist):
     dist = best_dist[0]
     args = dist.fit(var)
     distxl = best_dist[0].pdf(var, *args)
     st.kstest(var, dist.cdf, args)
     data1 = go.Scatter(x=var,y= (distxl * len(var) * var.max() / 1), mode= 'markers', name= best_dist[0].name)
     data2 = go.Histogram(x=var, nbinsx= 1, name='Valuación')
     data= [data1, data2]
     layout = go.Layout(title='Comparación con distribución obtenida', 
                        xaxis=dict(title=var.name),
                        yaxis=dict(title=''))
     fig = go.Figure(data=data,layout=layout)
     edes=pd.DataFrame(data=var.describe())
     edes.rename(columns={1:'Medidas descriptivas'},inplace=True)
     print(edes)
     return (iplot(fig, filename = 'Hist'))
dist(best_dist)

       Medidas descriptivas
count              7.00e+00
mean               1.45e+07
std                2.62e+06
min                1.23e+07
25%                1.26e+07
50%                1.41e+07
75%                1.49e+07
max                1.98e+07
